In [1]:
read_conll = lambda f: [
    [
        [(z.strip().split(" ")[0], z.strip().split(" ")[-1]) for z in y.strip().split("\n")]
        for y in x.strip().split("\n\n")
    ]
    for x in open(f).read().strip().split("-DOCSTART- -X- -X- O")
    if len(x.strip()) > 0
]

In [2]:
import json
from seqeval.scheme import Tokens, IOB2, Entities

In [3]:
def docs_to_sentences(docs, split):
    data = []

    for docid, sents in enumerate(docs):
        for sentid, sent in enumerate(sents):
            new_x = []
            for i, (t, l) in enumerate(sent):
                s, e = l.split("-") if l != "O" else (None, None)
                if s == "I" and (i == 0 or sent[i - 1][1] == "O" or sent[i - 1][1].split("-")[1] != e):
                    l = "B-" + l.split("-")[1]
                new_x.append((t, l))
            tokens, labels = tuple(zip(*new_x))
            new_x = {"id" : f"conll_{split}_{docid}_{sentid}", "tokens": tokens, "ner_tags": labels}
            data.append(new_x)

    Entities([x["ner_tags"] for x in data], IOB2)

    return data

In [10]:
for name, split in [("train", "train"), ("testa", "validation"), ("testb", "test")]:
    new_docs = read_conll(f"Identifying-Incorrect-Labels-In-CoNLL-2003/corrected_corpus/eng.{name}")
    old_docs = read_conll(f"Identifying-Incorrect-Labels-In-CoNLL-2003/original_corpus/eng.{name}")

    bad = 0
    matched_old_docs, matched_new_docs = [], []
    for o, n in zip(old_docs, new_docs):
        ## Has same chars
        assert "".join([t[0] for x in o for t in x]) == "".join([t[0] for x in n for t in x])
        
        if len(o) != len(n):
            bad += 1 
            continue 

        if any(" ".join([t[0] for t in x]) != " ".join([t[0] for t in y]) for x, y in zip(o, n)):
            bad += 1 
            continue

        matched_old_docs.append(o)
        matched_new_docs.append(n)

    matched_old_docs = docs_to_sentences(matched_old_docs, split)
    matched_new_docs = docs_to_sentences(matched_new_docs, split)

    with open(f"data/conll_corrected/{split}_original.jsonl", "w") as f:
        f.write("\n".join(list(map(json.dumps, matched_old_docs))))

    with open(f"data/conll_corrected/{split}_corrected.jsonl", "w") as f:
        f.write("\n".join(list(map(json.dumps, matched_new_docs))))

    diff_l = 0
    for o, n in zip(matched_old_docs, matched_new_docs):
        if o["ner_tags"] != n["ner_tags"]:
            diff_l += 1

    print(bad, diff_l, len(matched_new_docs), len(old_docs))

59 348 12528 946
15 234 2775 216
37 213 2581 231
